In [1]:
import numpy as np
import random

In [2]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]
    
print(getDeck())

[(9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3)]


In [3]:
### Shuffle the cards randomly. Each player gets 9 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3") 
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")  
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(12, 2), (12, 0), (11, 3), (14, 0), (10, 0), (14, 2), (10, 3), (13, 0)]
[(9, 0), (9, 2), (11, 2), (9, 3), (11, 1), (10, 1), (12, 3), (10, 2)]


In [4]:
class Player():
    def __init__(self, name):
        
        self.name = name
        self.cards = None
    
    ### -------------------------------------------------------------
    
    ### TO BE IMPLEMENTED - player's strategy 
    ### input: declared card, i.e., the card which is supposed
    ### to be the top card of the pile: If None - you can put any card you want because
    ### (a) it is the first turn (pile is empty) or (b) some cards were drawn in the previous turn)
    ### output: - player's true decision, player's declaration (if not equal - (s)he cheats)
    
    def putCard(self, declared_card):
        ### DO NOT REMOVE TRUE CARD cards.remove!!!
        ### return an object (not id): self.cards[id], not id
        ### for instance: return self.cards[0], self.cards[0] 
        ### IMPORTANT: If you want to draw cards instead of put, return "draw"
        ### for instance: return "draw" 
        return self.cards[0], self.cards[0] 
    
    ### TO BE IMPLEMENTED - Decide whether to check or not opponent's move (return True or False)
    def checkCard(self, opponent_declaration):
        pass
    
    ### Notification sent at the end of a round
    ### One may implement this method, capture data, and use it to get extra info
    ### -- checked = TRUE -> someone checked. If FALSE, the remaining inputs do not play any role
    ### -- iChecked = TRUE -> I decided to check my opponent (so it was my turn); 
    ###               FALSE -> my opponent checked and it was his turn
    ### -- iDrewCards = TRUE -> I drew cards (so I checked but was wrong or my opponent checked and was right); 
    ###                 FALSE -> otherwise
    ### -- revealedCard - some card (X, Y). Only if I checked.
    ### -- noTakenCards - number of taken cards
    def getCheckFeedback(self, checked, iChecked, iDrewCards, revealedCard, noTakenCards, log=True):
        if log: print("Feedback = " + self.name + " : checked this turn = " + str(checked) +
              "; I checked = " + str(iChecked) + "; I drew cards = " + 
                      str(iDrewCards) + "; revealed card = " + 
                      str(revealedCard) + "; number of taken cards = " + str(noTakenCards))
    
    
    ### -------------------------------------------------------------
    
    ### Init player's hand
    def startGame(self, cards):
        self.cards = cards
    
    ### Add some cards to player's hand (if (s)he checked opponent's move, but (s)he was wrong)
    def takeCards(self, cards_to_take):
        self.cards = self.cards + cards_to_take

In [5]:
# Some examplary random player

class RandomPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card
        
        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])
       
        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)             
            
        ### Yet, declared card should be no worse than a card on the top of the pile . 
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])
    

In [6]:
class Game():
    def __init__(self, players, log = True):
        self.players = players
        self.deck = getDeck()
        self.player_cards = getShuffled(self.deck)
        self.game_deck = self.player_cards[0] + self.player_cards[1]
        
        self.cheats = [0, 0]
        self.moves = [0, 0]
        self.checks = [0, 0]
        self.draw_decisions = [0, 0]
        
        for i, cards in zip([0, 1], self.player_cards):
            self.players[i].startGame(cards.copy())
            if log:
                print("Player (" + str(i + 1) + "): " + self.players[i].name + " received:")
                print(self.players[i].cards)
        
        ### Which card is on top
        self.true_card = None
        ### Which card was declared by active player
        self.declared_card = None
        
        ### Init pile: [-1] = top card
        self.pile = []
        
        ### Which player moves
        self.player_move = np.random.randint(2)
        
    def takeTurn(self, log = True):
        
        self.player_move = 1 - self.player_move
        
        if log: 
            print("")
            print("")
            print("==== CURRENT STATE ================================")
            print("==== " + self.players[self.player_move].name + " MOVES ====")
            print("Player (0): " + self.players[0].name + " hand:")
            print(self.players[0].cards)
            print("Player (1): " + self.players[1].name + " hand:")
            print(self.players[1].cards)
            print("Pile: ")
            print(self.pile)
            print("Declared top card:")
            print(self.declared_card)
            print("")
            
        activePlayer = self.players[self.player_move]
        opponent = self.players[1 - self.player_move]
        self.moves[self.player_move] += 1
        
        self.previous_declaration = self.declared_card
        decision = activePlayer.putCard(self.declared_card)
        
        if decision == "draw":
            
            if log: print("[+] " + activePlayer.name + " decides to draw cards")
            
            self.draw_decisions[self.player_move] += 1
            
            toTake = self.pile[max([-3, -len(self.pile)]):]
            for c in toTake: self.pile.remove(c)
            activePlayer.takeCards(toTake)
            for c in toTake: self.player_cards[self.player_move].append(c)
            
            self.declared_card = None
            self.true_card = None
            
            activePlayer.getCheckFeedback(False, False, False, None, None, log)
            opponent.getCheckFeedback(False, False, False, None, None, log)

        else:
            self.true_card, self.declared_card = decision
            if self.true_card != self.declared_card: self.cheats[self.player_move] += 1
            
            if log: print("[+] " + activePlayer.name + " puts " + str(self.true_card) +
                          " and declares " + str(self.declared_card))
        
            if not self.debugMove(): return False, self.player_move
        
            activePlayer.cards.remove(self.true_card)
            self.player_cards[self.player_move].remove(self.true_card) 
            self.pile.append(self.true_card)
        
            if opponent.checkCard(self.declared_card):
                
                self.checks[1 - self.player_move] += 1
                
                if log: print("[!] " + opponent.name + ": " + "I want to check")
                toTake = self.pile[max([-3, -len(self.pile)]):]
                for c in toTake: self.pile.remove(c)

                if not self.true_card == self.declared_card:
                    if log: print("\tYou are right!")
                    activePlayer.takeCards(toTake)
                
                    activePlayer.getCheckFeedback(True, False, True, None, len(toTake), log)
                    opponent.getCheckFeedback(True, True, False, tuple(toTake[-1]), len(toTake), log)
                
                    for c in toTake: self.player_cards[self.player_move].append(c)
                else:
                    if log: print("\tYou are wrong!")
                    opponent.takeCards(toTake)  
                
                    activePlayer.getCheckFeedback(True, False, False, None, len(toTake), log)
                    opponent.getCheckFeedback(True, True, True, tuple(toTake[-1]), len(toTake), log)
               
                    for c in toTake: self.player_cards[1 - self.player_move].append(c)
            
                if log:
                    print("Cards taken: ")
                    print(toTake)

                self.declared_card = None
                self.true_card = None
            else:
                activePlayer.getCheckFeedback(False, False, False, None, None, log)
                opponent.getCheckFeedback(False, False, False, None, None, log)

            
        if not self.debugGeneral(): return False, self.player_move
        return True, self.player_move
            
    def isFinished(self, log = True):
        if len(self.players[self.player_move].cards) == 0:
            if log: print(self.players[self.player_move].name + " wins!")
            return True
        return False

    def debugMove(self):
        if (self.previous_declaration is not None) and (self.true_card[0] < self.previous_declaration[0]) and \
                    len(self.players[self.player_move].cards) == 1:
            print("[ERROR] Last played card should be valid (it is revealed, you cannot cheat)!")
            return False
        if np.array(self.true_card).size != 2: 
            print("[ERROR] You put too many cards!")
            return False
        if self.true_card not in self.player_cards[self.player_move]:
            print("[ERROR] You do not have this card!")
            return False
        if self.true_card not in self.deck:
            print("[ERROR] There is no such card!")
            return False
        if (self.previous_declaration is not None) and len(self.pile) == 0:
            print("[ERROR] Inconsistency")
            return False
        if (self.previous_declaration is not None) and (self.declared_card[0] < self.previous_declaration[0]):
            print(len(self.pile))
            print(self.previous_declaration)
            print(self.declared_card)
            print(self.pile[-1])
            print("[ERROR] Improper move!")
            return False
        return True
    
    def debugGeneral(self):
        A = set(self.players[0].cards)
        B = set(self.players[1].cards)
        C = set(self.player_cards[0])
        D = set(self.player_cards[1])
        P = set(self.pile)
        E = set(self.game_deck)
        
        if not A == C: 
            print("Error 001")
            return False
        if not B == D:
            print("Error 002")
            return False
        if not A | B | P == E:
            print("Error 003")
            print(A)
            print(B)
            print(P)
            print(E)
            return False
        return True

Analyze few moves...

In [7]:
# player1 = SimplePlayer("Player A")
# player2 = DarekPlayer("Darek")
# game = Game([player1, player2])

# for i in range(50):
#     game.takeTurn()
#     if game.isFinished():
#         break

In [8]:
### Some debug players
class DrawPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        return "draw"
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([False, False])
    
class SimplePlayer(Player):
    
    ### player's simple strategy
    def putCard(self, declared_card):
#         print("MAM TERAZ", len(self.cards))
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
#             print("robię DRAW")
            return "draw"
        
        card = min(self.cards, key=lambda x: x[0])
        declaration = (card[0], card[1])
        if declared_card is not None:
            min_val = declared_card[0]
            if card[0] < min_val: declaration = (min(min_val + 1, 14), declaration[1])
        return card, declaration
    
    def checkCard(self, opponent_declaration):
        if opponent_declaration in self.cards: return True
        return np.random.choice([True, False], p=[0.3, 0.7])

## Our comparison player -- tego już nie ulepszamy

In [9]:
class OurComparisonPlayer(Player):
    def __init__(self, name):
        self.name = name
        self.pile_size = 0
        self.opponent_cards_num = 8
    
    def putCard(self, declared_card):
#         print("MAM TERAZ", len(self.cards))
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        # jak można cokolwiek to kładź najmniejszą i mów prawdę
        if declared_card is None:
            card = min(self.cards, key=lambda x: x[0])
            declaration = (card[0], card[1])
            return card, declaration
        
        # nie kłam jak masz co włożyć (ale 14 zostaje na koniec)
        if self.iHaveRightCard(declared_card):
            card = min(filter(lambda i: i[0] >= declared_card[0], self.cards.copy()))
            declaration = (card[0], card[1])
            return card, declaration
        
        card = min(self.cards, key=lambda x: x[0])
        declaration = (card[0], card[1])
        
        if declared_card is not None:
            min_val = declared_card[0]
            if card[0] < min_val:
                declaration = (min(min_val + 1, 14), declaration[1])
                
        return card, declaration
    
    def get_cards_without_14(self):
        my_cards_without_14 = self.cards.copy()
        max_card = max(my_cards_without_14, key=lambda x: x[0])
        if max_card[0] == 14:
            my_cards_without_14.remove(max_card)
        return my_cards_without_14
    
    def i_won(self):
        if len(self.cards) == 1 and self.cards[0][0] == 14:
            return True
        return False
    
    # sprawdz czy mam odpowiednią kartę, ale jedna 14 zostaje na koniec (tak jakby jej nie było)
    def iHaveRightCard(self, declared_card):
        #mam już tylko 14, więc ją wyłożę i wygram
        if self.i_won():
            return True
        
        my_cards_without_14 = self.get_cards_without_14()
        card = max(my_cards_without_14, key=lambda x: x[0])
        if card[0] < declared_card[0]:
            return False
        return True
    
    def getCheckFeedback(self, checked, iChecked, iDrewCards, revealedCard, noTakenCards, log=True):
        if log: print("Feedback = " + self.name + " : checked this turn = " + str(checked) +
              "; I checked = " + str(iChecked) + "; I drew cards = " + 
                      str(iDrewCards) + "; revealed card = " + 
                      str(revealedCard) + "; number of taken cards = " + str(noTakenCards))
    
    
    def checkCard(self, opponent_declaration):
        if opponent_declaration in self.cards:
            return True
        
        #nie warto ryzykować bo wygram w tym ruchu
        if self.i_won():
            return False
        
        if self.iHaveRightCard(opponent_declaration):
            return False
        
        return np.random.choice([True, False], p=[0.3, 0.7]) #TODO potestowac być może 0.2 0.8 beda lepsze

In [10]:
class BetterComparisonPlayer(Player):
    def __init__(self, name):
#         self.check_probs = [0.1, 0,9]
        self.name = name
        self.pile_size = 0
        self.opponent_cards_num = 8
        self.my_first_turn = True
        self.pile_possible_content = [] #tutaj dodajemy karty które włożyliśmy i które włożył przeciwnik (zakładamy że mowi prawdę), jeśli ktoś coś bierze, to usuwamy z tej listy
        self.my_putCard_history = [] #z tej listy nie kasujemy
        self.pile_certain_content = [] #tu wrzucamy to co położyliśmy ale też kasujemy jeśli ktoś bierze karty
        self.UNCERTAINTY = False
        self.i_just_put = False
        self.almost_lost = False #TODO wywalić, tylko dla statystyk czy strategia coś daje
#         print("-------------------")
    
    ### player's random strategy
    ##UWAGA zmieniłem tak żęby był tylko jeden return (i pilnujmy tego), bo przed return zapisujemy do historii
    def putCard(self, declared_card):
#         if self.my_first_turn:
#             self.my_first_turn = False
#             if declared_card is None: #zaczynam
#                 pass
#             else: #przeciwnik zaczyna, położył pierwszą kartę
#                 pass
        
        if self.UNCERTAINTY:  #ktoś draw albo nikt nie zrobił check ostatnim razem
#             print("UNCERTAINTY=TRUE")
            if declared_card is None: #przeciwnik zrobił draw (bo my nigdy nie robimy)
                #przybyła mu 1 2 albo 3 karty, ale trzeba wykminić ile (dlatego śledzimy pile_size)
                if self.pile_size == 1: #bierze naszą
                    self.pile_size = 0
                    self.opponent_cards_num += 1
#                     print("draw +1")
                    self.pile_possible_content = self.pile_certain_content[:-1]
                    self.pile_certain_content = self.pile_certain_content[:-1]
                elif self.pile_size == 2: #bierze naszą i swoją
                    self.pile_size = 0
                    self.opponent_cards_num += 2
#                     print("draw +2")
                    self.pile_possible_content = self.pile_certain_content[:-2]
                    self.pile_certain_content = self.pile_certain_content[:-1]
                else: #bierze naszą, swoją, naszą
                    self.pile_size -= 3
                    self.opponent_cards_num += 3
#                     print("draw +3")
                    self.pile_possible_content = self.pile_certain_content[:-3]
                    self.pile_certain_content = self.pile_certain_content[:-2]
                
            else: #nikt nie zrobił check, nie się nie zmieniło
#                 print("---")
                pass
        
        self.UNCERTAINTY = False
        
#         print("putCard przeciwnik ma", self.opponent_cards_num)
        
        #TODO usunąć to potem, nie będzie już draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            raise Exception("DRAW już nie powinien nigdy mieć miejsca")
            return "draw"
        
        # jak można cokolwiek to kładź najmniejszą i mów prawdę
        if declared_card is None:
            card = min(self.cards, key=lambda x: x[0])
            declaration = (card[0], card[1])
#             return card, declaration
        
        # nie kłam jak masz co włożyć (ale 14 zostaje na koniec)
        elif self.iHaveRightCard(declared_card):
            card = min(filter(lambda i: i[0] >= declared_card[0], self.cards.copy()))
            declaration = (card[0], card[1])
#             return card, declaration
        
        else: #oszukujemy
            card = min(self.cards, key=lambda x: x[0])
        
            if card[0] < declared_card[0]:
                declaration = self.goodCheatDeclaration(declared_card)
#                 print("Będę sciemniał że daję:", declaration)
#                 print("A naprawdę dam", card)
#                 declaration = (min(min_val + 1, 14), declaration[1])
        
        self.pile_possible_content.append(card)
        self.pile_certain_content.append(card)
        self.my_putCard_history.append(card)
        self.pile_size += 1
        
        self.i_just_put = True
        
#         if card == declaration and len(self.cards)==1:
# #             print("Wygrałem")
#             if self.almost_lost:
#                 print("wygrałem choć on miał już tylko 1 kartę")
        
        return card, declaration
    
    
    
    def get_cards_without_14(self):
        my_cards_without_14 = self.cards.copy()
        max_card = max(my_cards_without_14, key=lambda x: x[0])
        if max_card[0] == 14:
            my_cards_without_14.remove(max_card)
        return my_cards_without_14
    
    def i_won(self):
        if len(self.cards) == 1 and self.cards[0][0] == 14:
            return True
        return False
    
    # sprawdz czy mam odpowiednią kartę, ale jedna 14 zostaje na koniec (tak jakby jej nie było)
    def iHaveRightCard(self, declared_card):
        #mam już tylko 14, więc ją wyłożę i wygram
        if self.i_won():
            return True
        
        my_cards_without_14 = self.get_cards_without_14()
        card = max(my_cards_without_14, key=lambda x: x[0])
        if card[0] < declared_card[0]:
            return False
        return True
    
    def goodCheatDeclaration(self, declared_card):
#         print("Moje karty:", self.cards)
        
        if max(self.cards, key=lambda x: x[0])[0] >= declared_card[0]: #mogę kłamać deklarując jedną ze swoich
            declaration = min(filter(lambda i: i[0] >= declared_card[0], self.cards.copy()))
        else: #nie mam odpowiedniej karty
            #TODO zaimplemnetować żeby deklarować kartę która jeszcze się nie pojawiła w grze (trudne bo przeciwnik też kłamie)
            declaration = declared_card #TODO tylko tymczasowe, głupio deklarować to co przed chwilą przeciwnik, ale przeciwko Simple się sprawdzi
            
        return declaration
    
    def getCheckFeedback(self, checked, iChecked, iDrewCards, revealedCard, noTakenCards, log=True):
        #getCheckFeedback(False, False, False, None, None, log) -- wywoływane gdy ktoś draw albo nikt nie zrobił check, trzeba to rozróżnić
        if self.i_just_put:
            self.i_just_put = False
        else:
            if not checked:
#                 print("niepewność")
                self.UNCERTAINTY = True
        
        #getCheckFeedback(True, False, True, None, len(toTake), log) przeciwnik mnie przyłapał
        if checked and not iChecked and iDrewCards:
            for i in range(noTakenCards):
                self.pile_possible_content = self.pile_possible_content[:-1] #zdejmujemy odpowiednią ilość
            if noTakenCards == 3:
                self.pile_certain_content = self.pile_certain_content[:-2] #dwie z trzech ostatnich były nasze
            else:
                self.pile_certain_content = self.pile_certain_content[:-1] #jedna z dwóch ostatnich była nasza
        
        #getCheckFeedback(True, True, False, tuple(toTake[-1]), len(toTake), log) przyłapaliśmy przeciwnika
        if checked and iChecked and not iDrewCards:
            self.opponent_cards_num += noTakenCards
#             print("przyłapany:", noTakenCards)
            if noTakenCards > 1:
                self.pile_certain_content = self.pile_certain_content[:-1] #jedna była nasza
            for i in range(noTakenCards):
                self.pile_possible_content = self.pile_possible_content[:-1] #zdejmujemy odpowiednią ilość
                
        #getCheckFeedback(True, False, False, None, len(toTake), log) przeciwnik nie miał racji i bierze karty
        if checked and not iChecked and not iDrewCards:
            self.opponent_cards_num += noTakenCards
#             print("sam się wkopał:", noTakenCards)
            for i in range(noTakenCards-1):
                self.pile_certain_content = self.pile_certain_content[:-1] #bierze dwie lub jedną naszą
            for i in range(noTakenCards):
                self.pile_possible_content = self.pile_possible_content[:-1] #zdejmujemy odpowiednią ilość
        
        #getCheckFeedback(True, True, True, tuple(toTake[-1]), len(toTake), log) bierzemy karty bo nie mieliśmy racji
        if checked and iChecked and iDrewCards:
            if noTakenCards > 1:
                self.pile_certain_content = self.pile_certain_content[:-1] #jedna była nasza
            for i in range(noTakenCards):
                self.pile_possible_content = self.pile_possible_content[:-1] #zdejmujemy odpowiednią ilość
        
        if log: print("Feedback = " + self.name + " : checked this turn = " + str(checked) +
              "; I checked = " + str(iChecked) + "; I drew cards = " + 
                      str(iDrewCards) + "; revealed card = " + 
                      str(revealedCard) + "; number of taken cards = " + str(noTakenCards))
    
    
    def checkCard(self, opponent_declaration):
        self.opponent_cards_num -= 1
        self.pile_size += 1
        self.pile_possible_content.append(opponent_declaration)
        
#         if self.opponent_cards_num == 0:
#             print("Właśnie przegraliśmy")
        
#         print("checkCard przeciwnik ma: ", self.opponent_cards_num)

        #UWAGA tu ważna jest kolejność warunków
        
        #jesteśmy absolutnie pewni
        if opponent_declaration in self.cards:
            return True
        
        #nie warto ryzykować bo wygram w tym ruchu
        if self.i_won():
            return False 
        
        #zamiast robić draw w swoim ruchu
        if len(self.cards) == 1 and opponent_declaration is not None and self.cards[0][0] < opponent_declaration[0]:
            return True
        
        #przeciwnik zaraz wygra; TODO być może warto dodać dodatkowe warunki
        if self.opponent_cards_num == 1:
            if opponent_declaration[0] < 10: #nie wiem czy to dobra droga
#                 print("a może nie")
                return False
#             print("Zaraz wygra", self.opponent_cards_num)
            self.almost_lost = True
            return True
        
        if self.iHaveRightCard(opponent_declaration):#TODO jeśli on ma mało (dużo mniej od nas kart) to jednak może go sprawdzać z jakimś prawdopodobieństwem
            return False
        
        return np.random.choice([True, False], p=[0.3, 0.7]) #TODO potestowac być może 0.2 0.8 beda lepsze
    

#TODO test dwóch graczy, takich ze wszytkimi ulepszeniami, ale jeden nie będzie robił check kiedy przeciwnik ma ostatniąkartę
#TODO jeśli nie można oszukiwać jedną ze swoich kart to oszukujemy ostatnią deklaracją przeciwnika (to tylko chwilowy, słaby pomysł, zwłaszcza jeśli drugi gracz też oszukuje własnymi, wtedy wykryje to oszustwo)
#zapisywać karty które ma przeciwnik (wziął), nie oszukiwać że takie się ma
#przetestować czy poprawnie zaimplementowana liczność stosu i przewidywanie zawartości stosu
#zapisywać swoje oszustwa
#!!!TODO nie oszukiwać kartą którą wydaliśmy -- to warto zrobić
#na początku kłamać, kiedy on włożył(zadeklarował) większą od 9
#podbijamy jeśli nasza następna najmniejsza jest o 2 większa od bieżącej deklaracji

#nie wkładać swojej ostatniej 14
#TODO sprawidzić ^^^ (być może warto też zachować sobie 13, jeśli nie mamy żadnej 14)

#!!!TODO jeśli przeciwnik zadeklarował to co kiedyś włożyliśmy (a on tego nie wziął do siebie to robić checkCard)
#TODO jeśli przegrywamy to sprawdzać go losowo z większym prawdopodobieństwem, jeśli wygrywamy (zwłaszcza znacząco to go nie sprawdzać)

In [11]:
def run_n_games(player_1_class, player_2_class, repeats=1000):
    ### Perform a full game 100 times
    stats_wins = [0, 0]
    stats_moves = [0, 0]
    stats_cheats = [0, 0]
    stats_errors = [0, 0]
    stats_cards = [0, 0]
    stats_checks = [0, 0]
    stats_draw_decisions = [0, 0]
    stats_pile_size = 0

#     repeats = 1000
    errors = 0

    for t in range(repeats):
        player1 = player_1_class("Player A")
        player2 = player_2_class("Darek")
        game = Game([player1, player2], log = False)
#         print("----------------------------------------------- new game")

        error = False
        infinite_loop_counter = 0
        while True:
            if infinite_loop_counter > 10000:
                error = True
                errors += 1
                break
            valid, player = game.takeTurn(log = False)
            infinite_loop_counter += 1
            if not valid:
                error = True
                stats_errors[player] += 1
                errors += 1
                break
            if game.isFinished(log = False):
                stats_wins[player] += 1

    #             if player == 1:
    #                 print("Wygrał: ", player2.name)
                break

        stats_pile_size += len(game.pile)
        if not error:
            for j in range(2):
                stats_moves[j] += game.moves[j]
                stats_cheats[j] += game.cheats[j]
                stats_checks[j] += game.checks[j]
                stats_draw_decisions[j] += game.draw_decisions[j]
                stats_cards[j] += len(game.player_cards[j])

    stats_pile_size /= (repeats - errors)          
    for j in range(2):
        stats_moves[j] /= (repeats - errors)
        stats_cheats[j] /= (repeats - errors)
        stats_checks[j] /= (repeats - errors)
        stats_draw_decisions[j] /= (repeats - errors)
        stats_cards[j] /= (repeats - errors)


    print("Wins:")
    print(stats_wins)
    print("Moves:")
    print(stats_moves)
    print("Cards:")
    print(stats_cards)
    print("Pile size:")
    print(stats_pile_size)
    print("Checks:")
    print(stats_checks)
    print("Draw decisions:")
    print(stats_draw_decisions)
    print("Cheats:")
    print(stats_cheats)
    print("Errors:")
    print(stats_errors)
    print("Total errors:")
    print(errors)

In [12]:
class UltimateComparisonPlayer(Player):
    def __init__(self, name):
        self.name = "132235_132350"
        
        self.pile_size = 0
        self.opponent_cards_num = 8
        self.pile_certain_content = [] #tu wrzucamy to co położyliśmy ale też kasujemy jeśli ktoś bierze karty
        
        #zapisywanie historii stosu, do kogo należą kolejne karty, 0-karta przeciwnika, 1-moja
        #ponieważ nie zawsze karty moje i przeciwnika są na przemian (gdy ktoś weźmie może się to zmienić)
        self.pile_owners = []
        
        self.no_check_or_draw = False
        self.i_just_put = False
    
    
    
    
#     ----------------UTILS---------------------
    def get_cards_without_best(self):
        my_cards_without_best = self.cards.copy()
        max_card = max(my_cards_without_best, key=lambda x: x[0])
        if max_card[0] > 12:
            my_cards_without_best.remove(max_card)
        return my_cards_without_best
    
    #mam już tylko jedną i mogę ją włożyć więc wygram
    def i_won(self, opponent_declaration):
        if len(self.cards) == 1 and self.cards[0][0] >= opponent_declaration[0]:
            return True
        return False
    
    # sprawdz czy mam odpowiednią kartę, ale jedna 14 lub 13 zostaje na koniec (tak jakby jej nie było)
    def iHaveRightCard(self, declared_card):
        if self.i_won(declared_card):
            return True
        
        my_cards_without_best = self.get_cards_without_best()
        if len(my_cards_without_best) < 1:
            return False
        card = max(my_cards_without_best, key=lambda x: x[0])
        if card[0] < declared_card[0]:
            return False
        return True
    
    def goodCheatDeclaration(self, declared_card):
        if max(self.cards, key=lambda x: x[0])[0] >= declared_card[0]: #mogę kłamać deklarując jedną ze swoich
            declaration = min(filter(lambda i: i[0] >= declared_card[0], self.cards.copy()))
        else: #nie mam odpowiedniej karty
            declaration = declared_card
            if len(self.pile_certain_content) > 0 and max(self.pile_certain_content, key=lambda x: x[0])[0] >= declared_card[0]:
                declaration = min(filter(lambda i: i[0] >= declared_card[0], self.pile_certain_content.copy()))
        return declaration
#     ------------------------------------------
    
    
    ### player's random strategy
    ##UWAGA zmieniłem tak żęby był tylko jeden return (i pilnujmy tego), bo przed return zapisujemy do historii
    def putCard(self, declared_card):
        if self.no_check_or_draw and declared_card is None:
            #przybyła mu 1 2 albo 3 karty, ale trzeba wykminić ile (dlatego śledzimy pile_size)
            
            cards_to_draw = min(3, self.pile_size)
            self.pile_size -= cards_to_draw
            self.opponent_cards_num += cards_to_draw
            
            for i in range(cards_to_draw):
                if self.pile_owners[-1] == 1:
                    self.pile_certain_content = self.pile_certain_content[:-1]
                self.pile_owners = self.pile_owners[:-1]
        
        self.no_check_or_draw = False
        
        
        # jak można cokolwiek to kładź najmniejszą i mów prawdę
        if declared_card is None:
            card = min(self.cards, key=lambda x: x[0])
            declaration = (card[0], card[1])
        
        # nie kłam jak masz co włożyć (ale 14 zostaje na koniec)
        elif self.iHaveRightCard(declared_card):
            card = min(filter(lambda i: i[0] >= declared_card[0], self.cards.copy()))
            declaration = (card[0], card[1])
        
        else: #oszukujemy
            card = min(self.cards, key=lambda x: x[0])
            declaration = (card[0], card[1])
            
            if card[0] < declared_card[0]:
                declaration = self.goodCheatDeclaration(declared_card)
#                 print("Będę sciemniał że daję:", declaration)
#                 print("A naprawdę dam", card)
#                 declaration = (min(min_val + 1, 14), declaration[1])
        
        self.pile_certain_content.append(card)
        self.pile_owners.append(1)
        self.pile_size += 1
        self.i_just_put = True
        
        return card, declaration
    
    
    def getCheckFeedback(self, checked, iChecked, iDrewCards, revealedCard, noTakenCards, log=True):
        #getCheckFeedback(False, False, False, None, None, log)
        #wywoływane gdy ktoś draw albo nikt nie zrobił check, trzeba to rozróżnić
        if self.i_just_put: #ja właśnie włożyłem i nikt nie sprawdził
            self.i_just_put = False
        else: #przeciwnik włożył a ja nie sprawdziłem, albo przeciwnik zrobił draw
            if not checked:
                self.no_check_or_draw = True
                
        if checked:
            self.pile_size -= noTakenCards
            for i in range(noTakenCards):
                if self.pile_owners[-1] == 1:
                    self.pile_certain_content = self.pile_certain_content[:-1]
                self.pile_owners = self.pile_owners[:-1]
        
        #getCheckFeedback(True, False, True, None, len(toTake), log) przeciwnik mnie przyłapał
#         if checked and not iChecked and iDrewCards:
            
        
        #getCheckFeedback(True, True, False, tuple(toTake[-1]), len(toTake), log) przyłapaliśmy przeciwnika
        if checked and iChecked and not iDrewCards:
            self.opponent_cards_num += noTakenCards
                
        #getCheckFeedback(True, False, False, None, len(toTake), log) przeciwnik nie miał racji i bierze karty
        if checked and not iChecked and not iDrewCards:
            self.opponent_cards_num += noTakenCards

        
        #getCheckFeedback(True, True, True, tuple(toTake[-1]), len(toTake), log) bierzemy karty bo nie mieliśmy racji
#         if checked and iChecked and iDrewCards:
        
        if log: print("Feedback = " + self.name + " : checked this turn = " + str(checked) +
              "; I checked = " + str(iChecked) + "; I drew cards = " + 
                      str(iDrewCards) + "; revealed card = " + 
                      str(revealedCard) + "; number of taken cards = " + str(noTakenCards))
    
    
    def checkCard(self, opponent_declaration):
        self.opponent_cards_num -= 1
        self.pile_size += 1
        self.pile_owners.append(0)
        
        #jesteśmy absolutnie pewni
        if opponent_declaration in self.cards:
            return True
        
        #przeciwnik oszukuje kartą, którą wyłożyliśmy a on jej nie wziął, też 100% kłamstwo
        if opponent_declaration in self.pile_certain_content:
#             print("KŁAMSTWO: ", opponent_declaration, "jest w pile'u")
            return True

        #nie warto ryzykować bo wygram w tym ruchu
        if self.i_won(opponent_declaration):
            return False
        
        #zamiast robić draw w swoim ruchu
        if len(self.cards) == 1 and opponent_declaration is not None and self.cards[0][0] < opponent_declaration[0]:
            return True
        
        #włożył przedostatnią kartę, zaraz wygra
        if self.opponent_cards_num == 1 and opponent_declaration[0] > 9:
            return True
        
        if self.iHaveRightCard(opponent_declaration):#TODO jeśli on ma mało (dużo mniej od nas kart) to jednak może go sprawdzać z jakimś prawdopodobieństwem
            return False
        
#         if len(self.cards) < 3 and self.opponent_cards_num > 4:
#             return np.random.choice([True, False], p=[0.15, 0.85])
        
        return np.random.choice([True, False], p=[0.3, 0.7]) #TODO potestowac



#na początku kłamać, kiedy on włożył(zadeklarował) większą od 9
#podbijamy jeśli nasza następna najmniejsza jest o 2 większa od bieżącej deklaracji

#nie wkładać swojej ostatniej 14
#TODO sprawidzić ^^^ (być może warto też zachować sobie 13, jeśli nie mamy żadnej 14)

#TODO jeśli przegrywamy to sprawdzać go losowo z większym prawdopodobieństwem, jeśli wygrywamy (zwłaszcza znacząco to go nie sprawdzać)

## Ten poniżej jest nasz

#### Changelog względem comparisonPlayer'a [nawiasach wpływ na wynik]
- nie robimy już draw, jeśli została jedna to jest wykrywane wcześniej, w metodzie checkCard (i albo za karę weźmiemy 3 karty, albo przeciwnik weźmie, na wierzchu będzie None i wygrywamy [chyba trochę daje przewagę]
- oszukiwanie jedną ze swoich kart jeśli się da (albo deklaruje to co przeciwnik ostatnio) [daje przewagę z comparisonPlayer]
- dodane liczenie kart przeciwnika i check jeśli przeciwnik ma ostatnią [chyba daje przewagę z comparisonPlayer]

In [13]:
class YourPlayer(Player):
    def __init__(self, name):
        self.name = "132235_132350"
        
        self.pile_size = 0
        self.opponent_cards_num = 8
        self.pile_certain_content = [] #tu wrzucamy to co położyliśmy ale też kasujemy jeśli ktoś bierze karty
        
        #zapisywanie historii stosu, do kogo należą kolejne karty, 0-karta przeciwnika, 1-moja
        #ponieważ nie zawsze karty moje i przeciwnika są na przemian (gdy ktoś weźmie może się to zmienić)
        self.pile_owners = []
        
        self.no_check_or_draw = False
        self.i_just_put = False
    
    
#     ----------------UTILS---------------------
    def get_cards_without_best(self):
        my_cards_without_best = self.cards.copy()
        max_card = max(my_cards_without_best, key=lambda x: x[0])
        if max_card[0] > 12:
            my_cards_without_best.remove(max_card)
        return my_cards_without_best
    
    #mam już tylko jedną i mogę ją włożyć więc wygram
    def i_won(self, opponent_declaration):
        if len(self.cards) == 1 and self.cards[0][0] >= opponent_declaration[0]:
            return True
        return False
    
    # sprawdz czy mam odpowiednią kartę, ale jedna 14 lub 13 zostaje na koniec (tak jakby jej nie było)
    def iHaveRightCard(self, declared_card):
        if self.i_won(declared_card):
            return True
        
        my_cards_without_best = self.get_cards_without_best()
        if len(my_cards_without_best) < 1:
            return False
        card = max(my_cards_without_best, key=lambda x: x[0])
        if card[0] < declared_card[0]:
            return False
        return True
    
    def goodCheatDeclaration(self, declared_card):
        if max(self.cards, key=lambda x: x[0])[0] >= declared_card[0]: #mogę kłamać deklarując jedną ze swoich
            declaration = min(filter(lambda i: i[0] >= declared_card[0], self.cards.copy()))
        else: #nie mam odpowiedniej karty
            declaration = declared_card
            if len(self.pile_certain_content) > 0 and max(self.pile_certain_content, key=lambda x: x[0])[0] >= declared_card[0]:
                declaration = min(filter(lambda i: i[0] >= declared_card[0], self.pile_certain_content.copy()))
        return declaration
#     ------------------------------------------
    
    
    def putCard(self, declared_card):
        if self.no_check_or_draw and declared_card is None:
            #przybyła mu 1 2 albo 3 karty, ale trzeba wykminić ile (dlatego śledzimy pile_size)
            cards_to_draw = min(3, self.pile_size)
            self.pile_size -= cards_to_draw
            self.opponent_cards_num += cards_to_draw
            
            for i in range(cards_to_draw):
                if self.pile_owners[-1] == 1:
                    self.pile_certain_content = self.pile_certain_content[:-1]
                self.pile_owners = self.pile_owners[:-1]
        
        self.no_check_or_draw = False
        
        
        # jak można cokolwiek to kładź najmniejszą i mów prawdę
        if declared_card is None:
            card = min(self.cards, key=lambda x: x[0])
            declaration = (card[0], card[1])
        
        # nie kłam jak masz co włożyć (ale 14 zostaje na koniec)
        elif self.iHaveRightCard(declared_card):
            card = min(filter(lambda i: i[0] >= declared_card[0], self.cards.copy()))
            declaration = (card[0], card[1])
        
        else: #oszukujemy
            card = min(self.cards, key=lambda x: x[0])
            declaration = (card[0], card[1])
            
            if card[0] < declared_card[0]:
                declaration = self.goodCheatDeclaration(declared_card)
        
        self.pile_certain_content.append(card)
        self.pile_owners.append(1)
        self.pile_size += 1
        self.i_just_put = True
        
        return card, declaration
    
    
    def getCheckFeedback(self, checked, iChecked, iDrewCards, revealedCard, noTakenCards, log=True):
        #getCheckFeedback(False, False, False, None, None, log)
        #wywoływane gdy ktoś draw albo nikt nie zrobił check, trzeba to rozróżnić
        if self.i_just_put: #ja właśnie włożyłem i nikt nie sprawdził
            self.i_just_put = False
        else: #przeciwnik włożył a ja nie sprawdziłem, albo przeciwnik zrobił draw
            if not checked:
                self.no_check_or_draw = True
                
        if checked:
            self.pile_size -= noTakenCards
            for i in range(noTakenCards):
                if self.pile_owners[-1] == 1:
                    self.pile_certain_content = self.pile_certain_content[:-1]
                self.pile_owners = self.pile_owners[:-1]
        
        #getCheckFeedback(True, False, True, None, len(toTake), log) przeciwnik mnie przyłapał
#         if checked and not iChecked and iDrewCards:
            
        #getCheckFeedback(True, True, False, tuple(toTake[-1]), len(toTake), log) przyłapaliśmy przeciwnika
        if checked and iChecked and not iDrewCards:
            self.opponent_cards_num += noTakenCards
                
        #getCheckFeedback(True, False, False, None, len(toTake), log) przeciwnik nie miał racji i bierze karty
        if checked and not iChecked and not iDrewCards:
            self.opponent_cards_num += noTakenCards
        
        #getCheckFeedback(True, True, True, tuple(toTake[-1]), len(toTake), log) bierzemy karty bo nie mieliśmy racji
#         if checked and iChecked and iDrewCards:
        
        if log: print("Feedback = " + self.name + " : checked this turn = " + str(checked) +
              "; I checked = " + str(iChecked) + "; I drew cards = " + 
                      str(iDrewCards) + "; revealed card = " + 
                      str(revealedCard) + "; number of taken cards = " + str(noTakenCards))
    
    
    def checkCard(self, opponent_declaration):
        self.opponent_cards_num -= 1
        self.pile_size += 1
        self.pile_owners.append(0)
        
        #jesteśmy absolutnie pewni
        if opponent_declaration in self.cards:
            return True
        
        #przeciwnik oszukuje kartą, którą wyłożyliśmy a on jej nie wziął, też 100% kłamstwo
        if opponent_declaration in self.pile_certain_content:
#             print("KŁAMSTWO: ", opponent_declaration, "jest w pile'u")
            return True

        #nie warto ryzykować bo wygram w tym ruchu
        if self.i_won(opponent_declaration):
            return False
        
        #zamiast robić draw w swoim ruchu
        if len(self.cards) == 1 and opponent_declaration is not None and self.cards[0][0] < opponent_declaration[0]:
            return True
        
        #włożył przedostatnią kartę, zaraz wygra
        if self.opponent_cards_num == 1 and opponent_declaration[0] > 9:
            return True
        
        if self.iHaveRightCard(opponent_declaration):#TODO jeśli on ma mało (dużo mniej od nas kart) to jednak może go sprawdzać z jakimś prawdopodobieństwem
            return False
        
        #jak wygrywamy, to rzadziej sprawdzamy
        if len(self.cards) < 3 and self.opponent_cards_num > 4:
            return np.random.choice([True, False], p=[0.1, 0.9])
        
        return np.random.choice([True, False], p=[0.3, 0.7])

In [14]:
run_n_games(UltimateComparisonPlayer, YourPlayer, repeats=1000)
#te errory to nie są errory, tylko zrobiłem limit 10000 ruchów, żeby zapętlone gry nie zawieszały kernela

Wins:
[498, 502]
Moves:
[8.573, 8.571]
Cards:
[3.159, 3.113]
Pile size:
9.728
Checks:
[1.245, 1.227]
Draw decisions:
[0.0, 0.0]
Cheats:
[0.919, 0.931]
Errors:
[0, 0]
Total errors:
0


In [15]:
run_n_games(BetterComparisonPlayer, YourPlayer, repeats=1000)
#te errory to nie są errory, tylko zrobiłem limit 10000 ruchów, żeby zapętlone gry nie zawieszały kernela

Wins:
[487, 509]
Moves:
[8.545180722891565, 8.581325301204819]
Cards:
[3.25, 3.0240963855421685]
Pile size:
9.77008032128514
Checks:
[1.2028112449799198, 1.2640562248995983]
Draw decisions:
[0.0, 0.0]
Cheats:
[0.9106425702811245, 0.9246987951807228]
Errors:
[0, 0]
Total errors:
4


In [16]:
run_n_games(OurComparisonPlayer, YourPlayer, repeats=1000)

Wins:
[305, 695]
Moves:
[8.206, 8.406]
Cards:
[3.044, 1.816]
Pile size:
11.14
Checks:
[0.556, 1.259]
Draw decisions:
[0.007, 0.0]
Cheats:
[1.059, 0.743]
Errors:
[0, 0]
Total errors:
0


In [17]:
run_n_games(SimplePlayer, YourPlayer, repeats=1000)

Wins:
[55, 945]
Moves:
[8.395, 8.849]
Cards:
[9.879, 0.353]
Pile size:
5.768
Checks:
[2.723, 1.457]
Draw decisions:
[0.0, 0.0]
Cheats:
[3.372, 0.636]
Errors:
[0, 0]
Total errors:
0


In [18]:
run_n_games(RandomPlayer, YourPlayer, repeats=1000)

Wins:
[9, 991]
Moves:
[11.265, 11.771]
Cards:
[12.355, 0.069]
Pile size:
3.576
Checks:
[5.91, 1.765]
Draw decisions:
[0.002, 0.0]
Cheats:
[6.471, 2.221]
Errors:
[0, 0]
Total errors:
0


In [19]:
run_n_games(DrawPlayer, YourPlayer, repeats=100)

Wins:
[0, 100]
Moves:
[7.52, 8.0]
Cards:
[15.0, 0.0]
Pile size:
1.0
Checks:
[0.0, 0.0]
Draw decisions:
[7.52, 0.0]
Cheats:
[0.0, 0.0]
Errors:
[0, 0]
Total errors:
0
